In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s4e4/sample_submission.csv
/kaggle/input/playground-series-s4e4/train.csv
/kaggle/input/playground-series-s4e4/test.csv


In [2]:
train_dataset_root = r"/kaggle/input/playground-series-s4e4/train.csv"
test_dataset_root = r"/kaggle/input/playground-series-s4e4/test.csv"
train_dataset = pd.read_csv(train_dataset_root)
# one hot
train_dataset = pd.get_dummies(train_dataset, columns=['Sex'])
test_dataset = pd.read_csv(test_dataset_root)
test_dataset = pd.get_dummies(test_dataset, columns=['Sex'])
train_dataset.head(3)

,id,Length,Diameter,Height,Whole weight,Whole weight.1,Whole weight.2,Shell weight,Rings,Sex_F,Sex_I,Sex_M
0,0,0.55,0.43,0.150,0.7715,0.3285,0.1465,0.240,11,True,False,False
1,1,0.63,0.49,0.145,1.1300,0.4580,0.2765,0.320,11,True,False,False
2,2,0.16,0.11,0.025,0.0210,0.0055,0.0030,0.005,6,False,True,False


In [3]:
X = train_dataset.drop(columns=["id","Rings"]).values  # feature
y = train_dataset['Rings'].values  # label
print(X)
print(y)

[[0.55 0.43 0.15 ... True False False]
 [0.63 0.49 0.145 ... True False False]
 [0.16 0.11 0.025 ... False True False]
 ...
 [0.435 0.33 0.095 ... False True False]
 [0.345 0.27 0.075 ... False True False]
 [0.425 0.325 0.1 ... False True False]]
[11 11  6 ...  6  6  8]


# **Preprocess**

In [4]:
from sklearn.preprocessing import StandardScaler
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_train, X_val, y_train, y_val = train_test_split(X_scaled, y, test_size=0.2, random_state=42)
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val, dtype=torch.float32)
class DNN(nn.Module):
    def __init__(self, input_size, hidden_size1, hidden_size2):
        super(DNN, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size1)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size1, hidden_size2)
        self.fc3 = nn.Linear(hidden_size2, 1)

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        out = self.relu(out)
        out = self.fc3(out)
        return out

    
input_size = X_train.shape[1]
hidden_size1 = 64
hidden_size2 = 32
model = DNN(input_size, hidden_size1, hidden_size2)

criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 训练模型
num_epochs = 300
for epoch in range(num_epochs):
    # 前向传播
    outputs = model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor.view(-1, 1))

    # 反向传播和优化
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # 每10个epoch打印一次训练信息
    if epoch % 10 == 0:
        print(f'Epoch {epoch}, Loss: {loss.item():.4f}')
    
with torch.no_grad():
    val_outputs = model(X_val_tensor)
    val_loss = criterion(val_outputs, y_val_tensor.view(-1, 1))
    print(f'Validation Loss: {val_loss.item():.4f}')


Epoch 0, Loss: 102.6925
Epoch 10, Loss: 96.4140
Epoch 20, Loss: 88.3230
Epoch 30, Loss: 77.4351
Epoch 40, Loss: 63.8756
Epoch 50, Loss: 48.9241
Epoch 60, Loss: 35.1072
Epoch 70, Loss: 25.0386
Epoch 80, Loss: 20.1985
Epoch 90, Loss: 17.9004
Epoch 100, Loss: 15.3662
Epoch 110, Loss: 13.0087
Epoch 120, Loss: 10.9906
Epoch 130, Loss: 9.2781
Epoch 140, Loss: 7.9093
Epoch 150, Loss: 6.8996
Epoch 160, Loss: 6.1943
Epoch 170, Loss: 5.7161
Epoch 180, Loss: 5.3981
Epoch 190, Loss: 5.1832
Epoch 200, Loss: 5.0292
Epoch 210, Loss: 4.9111
Epoch 220, Loss: 4.8142
Epoch 230, Loss: 4.7307
Epoch 240, Loss: 4.6564
Epoch 250, Loss: 4.5885
Epoch 260, Loss: 4.5264
Epoch 270, Loss: 4.4695
Epoch 280, Loss: 4.4175
Epoch 290, Loss: 4.3699
Validation Loss: 4.3694


In [5]:
X = test_dataset.drop(columns=["id"]).values
test_id = test_dataset["id"].values
print(X)
test_scaled = scaler.fit_transform(X)
test_tensor = torch.tensor(test_scaled, dtype=torch.float32)
with torch.no_grad():
    test_outputs = model(test_tensor).numpy()
result_df = pd.DataFrame({'id': test_id, 'Rings': test_outputs.flatten()})
# 将DataFrame保存到CSV文件中
result_df.to_csv('submission.csv', index=False)

[[0.645 0.475 0.155 ... False False True]
 [0.58 0.46 0.16 ... False False True]
 [0.56 0.42 0.14 ... False False True]
 ...
 [0.59 0.44 0.155 ... False True False]
 [0.66 0.525 0.19 ... True False False]
 [0.43 0.34 0.12 ... True False False]]
